### API 활용_스타벅스 매장관련 데이터 분석


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

In [2]:
def get_seoul():
    
    url = 'https://www.starbucks.co.kr/store/getStore.do?r=UENRAT44P1'
    data = {
    'ins_lat':'37.2637696',
    'ins_lng':'127.0939648',
    'p_sido_cd':'01', 
    'p_gugun_cd':'',  
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000'
    }
    response = requests.post(url, data)
    temp_list = response.json()['list']
    df_temp = pd.DataFrame(temp_list)
    
    df = df_temp[['sido_name', 'gugun_name', 's_name', 'addr', 'doro_address','open_dt', 'lat', 'lot' ]]
    df = df.sort_values('open_dt', ascending = True).reset_index(drop=True)
    df.columns = ['시/도', '구/군', '매장명', '주소', '도로명 주소', '오픈일', '경도', '위도']
    
    df['위도'] = df['위도'].astype(float)
    df['경도'] = df['경도'].astype(float)
    
    file_name = 'starbucks_seoul.csv'
    df.to_csv(file_name, index=False)
    df_read = pd.read_csv(file_name)
        
    return df

In [3]:
df = get_seoul()
df

,시/도,구/군,매장명,주소,도로명 주소,오픈일,경도,위도
0,서울,서대문구,이대R,서울특별시 서대문구 대현동 54-4,서울특별시 서대문구 이화여대길 34 (대현동),19990727,37.558483,126.945964
1,서울,영등포구,여의도,서울특별시 영등포구 여의도동 23-9,서울특별시 영등포구 국제금융로2길 28 (여의도동),20001009,37.524035,126.924518
2,서울,강남구,압구정역,서울특별시 강남구 신사동 580-2 관영빌딩 1층 지하1층,"서울특별시 강남구 압구정로 170, 관영빌딩 1층 지하1층 (신사동)",20010310,37.526741,127.028110
3,서울,성동구,한양대,"서울특별시 성동구 행당동 31-11 경희빌딩 1,2층","서울특별시 성동구 왕십리로 225 (행당동) 1,2층",20010420,37.557270,127.042165
4,서울,종로구,인사,서울특별시 종로구 인사동 39,서울특별시 종로구 인사동길 14 (인사동),20010810,37.572129,126.987035
...,...,...,...,...,...,...,...,...
594,서울,강남구,언주역,서울특별시 강남구 역삼동 652-2,서울특별시 강남구 봉은사로 222 (역삼동),20221124,37.507775,127.036472
595,서울,서대문구,이대역,서울특별시 서대문구 대현동 45-8 유인빌딩,서울특별시 서대문구 신촌로 183 (대현동),20221213,37.557086,126.946280
596,서울,성동구,신금호역,서울특별시 성동구 금호동1가 122 1~5층 전체,서울특별시 성동구 금호로 158-1 (금호동1가) 1~5층 전체,20221213,37.554967,127.020145
597,서울,동대문구,경동1960,서울특별시 동대문구 제기동 1036경동시장 3~4층,서울특별시 동대문구 고산자로36길 3 (제기동),20221216,37.579614,127.038685


In [139]:
df.dtypes

시/도        object
구/군        object
매장명        object
주소         object
도로명 주소     object
오픈일        object
경도        float64
위도        float64
dtype: object

### 스타벅스 전국 매장 스크래핑

In [55]:
def get_all(sido_code):
    
    url = 'https://www.starbucks.co.kr/store/getStore.do?r=UENRAT44P1'
    data = {
    'ins_lat':'37.2637696',
    'ins_lng':'127.0939648',
    'p_sido_cd':sido_code, 
    'p_gugun_cd':'',  
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000'
    }
    response = requests.post(url, data)
    temp_list = response.json()['list']
    df_temp = pd.DataFrame(temp_list)
    
    df = df_temp[['sido_name', 'gugun_name', 's_name', 'addr', 'doro_address','open_dt', 'lat', 'lot', 'sido_code' ]]
    df = df.sort_values('open_dt', ascending = True).reset_index(drop=True)
    df.columns = ['시/도', '구/군', '매장명', '주소', '도로명 주소', '오픈일', '경도', '위도', '코드']
    
    df['위도'] = df['위도'].astype(float)
    df['경도'] = df['경도'].astype(float)
    
    file_name = 'starbucks_all.csv'
    df.to_csv(file_name, index=False)
    df_read = pd.read_csv(file_name)
        
    return df

In [56]:
temp_list = [] 

for no in range(1, 18):
    
    if no <= 9:
        df_temp = get_all(f'0{no}')
    else:
        df_temp = get_all(no)
    
    temp_list.append(df_temp)
    time.sleep(0.01)
        
df = pd.concat(temp_list).reset_index(drop=True)
df

,시/도,구/군,매장명,주소,도로명 주소,오픈일,경도,위도,코드
0,서울,서대문구,이대R,서울특별시 서대문구 대현동 54-4,서울특별시 서대문구 이화여대길 34 (대현동),19990727,37.558483,126.945964,01
1,서울,영등포구,여의도,서울특별시 영등포구 여의도동 23-9,서울특별시 영등포구 국제금융로2길 28 (여의도동),20001009,37.524035,126.924518,01
2,서울,강남구,압구정역,서울특별시 강남구 신사동 580-2 관영빌딩 1층 지하1층,"서울특별시 강남구 압구정로 170, 관영빌딩 1층 지하1층 (신사동)",20010310,37.526741,127.028110,01
3,서울,성동구,한양대,"서울특별시 성동구 행당동 31-11 경희빌딩 1,2층","서울특별시 성동구 왕십리로 225 (행당동) 1,2층",20010420,37.557270,127.042165,01
4,서울,종로구,인사,서울특별시 종로구 인사동 39,서울특별시 종로구 인사동길 14 (인사동),20010810,37.572129,126.987035,01
...,...,...,...,...,...,...,...,...,...
1774,세종,None,세종보람,세종특별자치시 보람동 700-81 호려울마을7단지 근린생활시설,"세종특별자치시 한누리대로 2165 (보람동) A118,119,120,121,128,...",20191212,36.477332,127.286519,17
1775,세종,None,세종소담,"세종특별자치시 소담동 539 새샘마을7단지 근린생활시설 -1동 12호,13호,14호...","세종특별자치시 소담3로 8 (소담동) 1동 12호,13호,14호,15호,15a호,16호",20210115,36.485755,127.300509,17
1776,세종,None,세종아름,"세종특별자치시 아름동 1329 다올비즈니스센터 103,104,105,202,203호","세종특별자치시 아름서1길 13-9 (아름동) 103,104,105,202,203호",20210226,36.511880,127.242770,17
1777,세종,None,세종충남대학교병원,세종특별자치시 도담동 407 세종충남대학교병원 헬스케어센터 1층,세종특별자치시 보듬7로 20 (도담동) 헬스케어센터 1층,20210609,36.519360,127.258280,17


In [57]:
df_code = pd.DataFrame(index=df['시/도'].unique())
df_code = df_code.drop('강원도')
df_code['코드'] = df['코드'].unique()
df_code

,코드
서울,01
광주,02
대구,03
대전,04
부산,05
울산,06
인천,07
경기,08
강원,09
경남,10
